#### План проекта

В рамках данной работы планируется реализовать систему оптимального показа рекламных баннеров на основе пользовательской активности, зафиксированной в логах `actions.csv` (показы и клики) и `triggers.csv` (посещения сайтов). Цель — максимизировать прибыль при соблюдении бизнес-ограничений: стоимость показа баннера составляет *1\$*, прибыль с клика — *5\$*, одному пользователю можно показывать рекламу не чаще одного раза в 14 дней.

На первом этапе будет проведена загрузка и предварительная обработка данных, включая фильтрацию по пересечению пользователей между двумя таблицами и, при необходимости, ограничение выборки (например, 100 000 пользователей для тестирования). Далее, для каждого показа будут сформированы поведенческие признаки, описывающие активность пользователя перед моментом взаимодействия. В качестве таких признаков будут использоваться: частота и уникальность посещений за разные интервалы времени, временные характеристики последнего посещения, история взаимодействия с рекламой и рассчитанный пользовательский CTR.

После построения фичей планируется обучение модели машинного обучения (например, LightGBM) для предсказания вероятности клика. В связи с сильным дисбалансом классов (около 3% положительных примеров), будет применено стратифицированное разбиение и метод SMOTE, что позволит улучшить возможность модели различать интересующих нас пользователей и снизить риск игнорирования положительного класса. На основе выходов модели будет реализован жадный алгоритм показа: реклама будет показана только тем пользователям, для которых уровень вероятности клика превысит минимальный для окупаемости.

Финальная часть проекта включает имитацию кампании с учетом 14-дневного ограничения на частоту показа, расчёт ключевых бизнес-метрик (показов, кликов, расходов, выручки и прибыли) и анализ эффективности выбранного порога.

#### 1. Загрузка, анализ и предварительная обработка данных

In [62]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

In [61]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
# Загрузим имеющиеся наборы данных
actions_df = pd.read_csv('actions.csv', parse_dates=['date'])
triggers_df = pd.read_csv('triggers.csv', parse_dates=['date'])

In [65]:
# Оценим размеры датасетов
print(f'Количество данных: \nactions_df – {actions_df.shape} \ntriggers_df – {triggers_df.shape}')

Количество данных: 
actions_df – (378204, 3) 
triggers_df – (43074627, 4)


In [66]:
# Пример данных
actions_df.head(3)

,guid,date,result
0,0187a45c-6784-7e2f-5d84-f3c89dee6a60,2024-05-20 08:28:13,0
1,0187a45d-650b-4a4f-ea59-9432556c9b1d,2024-05-31 08:19:10,0
2,018ba1bd-3c62-0269-e77f-655655f10b3e,2024-05-13 09:01:37,0


In [67]:
actions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378204 entries, 0 to 378203
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   guid    378204 non-null  object        
 1   date    378204 non-null  datetime64[ns]
 2   result  378204 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 8.7+ MB


In [68]:
# Проверим, нет ли пропусков в данных
print('Пропуски в actions_df:')
print(actions_df.isnull().sum().sort_values(ascending=False))
print('\nПропуски в triggers_df:')
print(triggers_df.isnull().sum().sort_values(ascending=False))

Пропуски в actions_df:
guid      0
date      0
result    0
dtype: int64

Пропуски в triggers_df:
guid       0
date       0
trigger    0
type       0
dtype: int64


In [69]:
# Проверим распределение таргета 
print(actions_df['result'].value_counts(normalize=True))

result
0    0.97055
1    0.02945
Name: proportion, dtype: float64


Так как целевая переменная _(result)_ есть только в actions.csv, разумно ограничиться только пользователями, у которых есть хоть один action, то есть выбираем из triggers.csv только те guid, которые есть в actions.csv

In [70]:
# Получение уникальных пользователей, у которых были взаимодействия
action_guids = actions_df['guid'].unique()

В целях увеличения скорости работы и оптимизации загрузки памяти, ограничимся 100k пользователей

In [71]:
# Случайная выборка 100_000 пользователей из тех, кто взаимодействовал
sample_guids = np.random.choice(action_guids, size=100_000, replace=False)

In [72]:
# Отфильтруем данные по выбранным пользователям
actions_small_df = actions_df[actions_df['guid'].isin(sample_guids)].copy()
triggers_small_df = triggers_df[triggers_df['guid'].isin(sample_guids)].copy()

In [73]:
# Выполним сортировку по пользователю и дате
actions_small_df = actions_small_df.sort_values(by=['guid', 'date'])
triggers_small_df = triggers_small_df.sort_values(by=['guid', 'date'])

#### 2. Генерация признаков

Временные признаки из `triggers`

In [74]:
# Последняя дата посещения для каждого пользователя
last_trigger_date = triggers_small_df.groupby('guid')['date'].max().reset_index()
last_trigger_date.rename(columns={'date': 'last_trigger_date'}, inplace=True)

In [75]:
# Количество дней с момента последнего посещения
current_date_triggers = triggers_small_df['date'].max()
last_trigger_date['days_since_last_trigger'] = (current_date_triggers - last_trigger_date['last_trigger_date']).dt.days

In [76]:
# Общее количество посещений
trigger_counts = triggers_small_df.groupby('guid').size().reset_index(name='total_triggers')

In [77]:
# Количество уникальных ресурсов, которые посещал пользователь
unique_triggers_count = triggers_small_df.groupby('guid')['trigger'].nunique().reset_index(name='unique_trigger_ids')

In [78]:
# Час и день недели последнего посещения
triggers_small_df['hour_of_day'] = triggers_small_df['date'].dt.hour
triggers_small_df['day_of_week'] = triggers_small_df['date'].dt.dayofweek

last_trigger_time_features = triggers_small_df.groupby('guid').tail(1)[['guid', 'hour_of_day', 'day_of_week']]
last_trigger_time_features.rename(columns={
    'hour_of_day': 'hour_of_day_last_trigger',
    'day_of_week': 'day_of_week_last_trigger'
}, inplace=True)

Поведенческие признаки из `actions`

In [79]:
# Количество кликов и показов рекламы
past_ad_interactions = actions_small_df.groupby('guid').agg(
    past_ad_clicks=('result', lambda x: (x == 1).sum()),
    past_ad_impressions=('result', 'count')
).reset_index()

In [80]:
# Обработка деления на ноль
past_ad_interactions['user_historical_ctr'] = past_ad_interactions.apply(
    lambda row: row['past_ad_clicks'] / row['past_ad_impressions'] if row['past_ad_impressions'] > 0 else 0,
    axis=1
)

In [81]:
# Дата последнего показа рекламы
last_ad_impression_date = actions_small_df.groupby('guid')['date'].max().reset_index()
last_ad_impression_date.rename(columns={'date': 'last_ad_impression_date'}, inplace=True)

In [82]:
# Количество дней с момента последнего показа
current_date_actions = actions_small_df['date'].max()
last_ad_impression_date['days_since_last_ad_impression'] = (
    current_date_actions - last_ad_impression_date['last_ad_impression_date']
).dt.days

RFM признаки (адаптированные)

In [83]:
# R (давность): минимум из дней с момента последнего показа и последнего посещения
rfm_recency = pd.merge(last_trigger_date[['guid', 'days_since_last_trigger']],
                       last_ad_impression_date[['guid', 'days_since_last_ad_impression']],
                       on='guid', how='outer')
rfm_recency['rfm_recency_score'] = rfm_recency[['days_since_last_trigger', 'days_since_last_ad_impression']].min(axis=1)

In [84]:
# F (частота): сумма всех показов и посещений
rfm_frequency = pd.merge(trigger_counts, past_ad_interactions[['guid', 'past_ad_impressions']],
                         on='guid', how='outer').fillna(0)
rfm_frequency['rfm_frequency_score'] = rfm_frequency['total_triggers'] + rfm_frequency['past_ad_impressions']

In [85]:
# M (ценность): сумма всех кликов и нормализованное число уникальных триггеров
rfm_engagement = pd.merge(past_ad_interactions[['guid', 'past_ad_clicks']],
                          unique_triggers_count, on='guid', how='outer').fillna(0)
rfm_engagement['rfm_engagement_score'] = (
    rfm_engagement['past_ad_clicks']
    + (rfm_engagement['unique_trigger_ids'] / rfm_engagement['unique_trigger_ids'].max())
)

Объединение всех признаков 

In [86]:
user_features = pd.DataFrame(sample_guids, columns=['guid'])

user_features = pd.merge(user_features, last_trigger_date[['guid', 'days_since_last_trigger']], on='guid', how='left')
user_features = pd.merge(user_features, trigger_counts, on='guid', how='left')
user_features = pd.merge(user_features, unique_triggers_count, on='guid', how='left')
user_features = pd.merge(user_features, last_trigger_time_features, on='guid', how='left')
user_features = pd.merge(user_features, past_ad_interactions, on='guid', how='left')
user_features = pd.merge(user_features, last_ad_impression_date, on='guid', how='left')
user_features = pd.merge(user_features, rfm_recency[['guid', 'rfm_recency_score']], on='guid', how='left')
user_features = pd.merge(user_features, rfm_frequency[['guid', 'rfm_frequency_score']], on='guid', how='left')
user_features = pd.merge(user_features, rfm_engagement[['guid', 'rfm_engagement_score']], on='guid', how='left')

# Замена пропусков на 0
user_features = user_features.fillna(0)

#### 3. Построение модели

In [88]:
# Получение метки (result) из последнего действия пользователя
user_latest_action = actions_small_df.sort_values('date').groupby('guid').tail(1).reset_index(drop=True)
modeling_df = pd.merge(user_features, user_latest_action[['guid', 'result']], on='guid', how='inner')

In [89]:
# Подготовка данных
X = modeling_df.drop(columns=['guid', 'last_ad_impression_date', 'result'])
y = modeling_df['result']

In [90]:
# Масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [91]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [93]:
# Балансировка классов через SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [95]:
# Обучение модели LightGBM
lgb_clf = lgb.LGBMClassifier(random_state=42, class_weight='balanced')
lgb_clf.fit(X_train_resampled, y_train_resampled)

[LightGBM] [Info] Number of positive: 77815, number of negative: 77815
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005273 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2863
[LightGBM] [Info] Number of data points in the train set: 155630, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


LGBMClassifier(class_weight='balanced', random_state=42)

In [96]:
# Предсказания на тестовой выборке
y_pred_proba = lgb_clf.predict_proba(X_test)[:, 1]
y_pred = lgb_clf.predict(X_test)

In [97]:
# Метрики качества
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

print(f'Оценка модели:')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'AUC-ROC: {roc_auc:.4f}')
print(f'AUC-PR: {pr_auc:.4f}')

Оценка модели:
Precision: 0.7776
Recall: 0.9158
F1-score: 0.8410
AUC-ROC: 0.9987
AUC-PR: 0.9601


#### 4. Оптимизация показа рекламы при положительном балансе

In [98]:
# Стоимость и вознаграждение
cost_per_impression = 1
reward_per_conversion = 5
profit_per_conversion = reward_per_conversion - cost_per_impression

In [141]:
# Минимальный уровень вероятности клика для окупаемости:
# P*5 - 1 > 0 => P > 0.2
break_even_ctr = 0.20

In [136]:
# Предсказание вероятностей клика для всех пользователей
X_all_users = X_scaled
all_user_guids = modeling_df['guid']
predicted_probabilities = lgb_clf.predict_proba(X_all_users)[:, 1]

predictions_df = pd.DataFrame({
    'guid': all_user_guids,
    'predicted_p_click': predicted_probabilities
})

In [137]:
# Отбор пользователей с ожидаемой положительной прибылью
profitable_users = predictions_df[predictions_df['predicted_p_click'] > break_even_ctr].copy()

In [138]:
# Применение ограничения на частоту показов — не чаще одного раза в 14 дней
profitable_users_with_recency = pd.merge(
    profitable_users,
    user_features[['guid', 'days_since_last_ad_impression']],
    on='guid',
    how='left'
)

eligible_users_for_ad = profitable_users_with_recency[
    (profitable_users_with_recency['days_since_last_ad_impression'] >= 14) |
    (profitable_users_with_recency['days_since_last_ad_impression'].isna())
].copy()

In [139]:
# Сортировка по вероятности клика для приоритезации
eligible_users_for_ad = eligible_users_for_ad.sort_values(by='predicted_p_click', ascending=False)

In [140]:
# Симуляция показов
total_impressions = 0
total_conversions = 0
total_cost = 0
total_revenue = 0

for index, row in eligible_users_for_ad.iterrows():
    total_impressions += 1
    total_cost += cost_per_impression
    
    if np.random.rand() < row['predicted_p_click']:
        total_conversions += 1
        total_revenue += reward_per_conversion
    
    current_balance = total_revenue - total_cost
    if current_balance < 0:
        total_impressions -= 1
        total_cost -= cost_per_impression
        if np.random.rand() < row['predicted_p_click']:
            total_revenue -= reward_per_conversion
        break

final_net_profit = total_revenue - total_cost

print(f'Результаты оптимизации:')
print(f'Всего показов: {total_impressions}')
print(f'Смоделированных конверсий: {total_conversions}')
print(f'Общие затраты: ${total_cost}')
print(f'Общий доход: ${total_revenue}')
print(f'Итоговая прибыль: ${final_net_profit}')

Результаты оптимизации:
Всего показов: 1313
Смоделированных конверсий: 1273
Общие затраты: $1313
Общий доход: $6365
Итоговая прибыль: $5052
